# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 8 2022 8:46AM,239069,10,NST-1047,"Amcyte Pharma, Inc.",Released
1,Jun 8 2022 8:46AM,239069,10,NST-1048,"Amcyte Pharma, Inc.",Released
2,Jun 8 2022 8:45AM,239067,10,0085745567,ISDIN Corporation,Released
3,Jun 8 2022 8:45AM,239067,10,0085745596,ISDIN Corporation,Released
4,Jun 8 2022 8:45AM,239067,10,0085745597,ISDIN Corporation,Released
5,Jun 8 2022 8:45AM,239067,10,0085745613,ISDIN Corporation,Released
6,Jun 8 2022 8:45AM,239067,10,0085745615,ISDIN Corporation,Released
7,Jun 8 2022 8:45AM,239067,10,0085745622,ISDIN Corporation,Released
8,Jun 8 2022 8:45AM,239067,10,0085745626,ISDIN Corporation,Released
9,Jun 8 2022 8:45AM,239067,10,0085745641,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,239065,Released,1
43,239066,Released,1
44,239067,Released,26
45,239068,Released,1
46,239069,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
239065,NaN,NaN,1.0
239066,NaN,NaN,1.0
239067,NaN,NaN,26.0
239068,NaN,NaN,1.0
239069,NaN,NaN,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,0.0,9.0,8.0
238956,3.0,18.0,1.0
238958,0.0,0.0,7.0
238984,0.0,1.0,1.0
238989,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238942,0,9,8
238956,3,18,1
238958,0,0,7
238984,0,1,1
238989,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,0,9,8
1,238956,3,18,1
2,238958,0,0,7
3,238984,0,1,1
4,238989,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238942,,9,8
1,238956,3,18,1
2,238958,,,7
3,238984,,1,1
4,238989,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc."
2,Jun 8 2022 8:45AM,239067,10,ISDIN Corporation
28,Jun 8 2022 8:45AM,239068,22,"NBTY Global, Inc."
29,Jun 8 2022 8:40AM,239066,19,"GCH Granules USA, Inc."
30,Jun 8 2022 8:28AM,239065,19,"GCH Granules USA, Inc."
31,Jun 8 2022 8:19AM,239064,12,Hush Hush
32,Jun 8 2022 8:12AM,239063,19,ACG North America LLC
34,Jun 8 2022 7:58AM,239062,10,"Methapharm, Inc."
35,Jun 8 2022 7:55AM,239061,22,"NBTY Global, Inc."
36,Jun 8 2022 7:52AM,239060,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",,,2
1,Jun 8 2022 8:45AM,239067,10,ISDIN Corporation,,,26
2,Jun 8 2022 8:45AM,239068,22,"NBTY Global, Inc.",,,1
3,Jun 8 2022 8:40AM,239066,19,"GCH Granules USA, Inc.",,,1
4,Jun 8 2022 8:28AM,239065,19,"GCH Granules USA, Inc.",,,1
5,Jun 8 2022 8:19AM,239064,12,Hush Hush,,1,
6,Jun 8 2022 8:12AM,239063,19,ACG North America LLC,,,2
7,Jun 8 2022 7:58AM,239062,10,"Methapharm, Inc.",,,1
8,Jun 8 2022 7:55AM,239061,22,"NBTY Global, Inc.",,,1
9,Jun 8 2022 7:52AM,239060,19,"AdvaGen Pharma, Ltd",,,35


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",2,,
1,Jun 8 2022 8:45AM,239067,10,ISDIN Corporation,26,,
2,Jun 8 2022 8:45AM,239068,22,"NBTY Global, Inc.",1,,
3,Jun 8 2022 8:40AM,239066,19,"GCH Granules USA, Inc.",1,,
4,Jun 8 2022 8:28AM,239065,19,"GCH Granules USA, Inc.",1,,
5,Jun 8 2022 8:19AM,239064,12,Hush Hush,,1,
6,Jun 8 2022 8:12AM,239063,19,ACG North America LLC,2,,
7,Jun 8 2022 7:58AM,239062,10,"Methapharm, Inc.",1,,
8,Jun 8 2022 7:55AM,239061,22,"NBTY Global, Inc.",1,,
9,Jun 8 2022 7:52AM,239060,19,"AdvaGen Pharma, Ltd",35,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",2,,
1,Jun 8 2022 8:45AM,239067,10,ISDIN Corporation,26,,
2,Jun 8 2022 8:45AM,239068,22,"NBTY Global, Inc.",1,,
3,Jun 8 2022 8:40AM,239066,19,"GCH Granules USA, Inc.",1,,
4,Jun 8 2022 8:28AM,239065,19,"GCH Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",2.0,NaN,NaN
1,Jun 8 2022 8:45AM,239067,10,ISDIN Corporation,26.0,NaN,NaN
2,Jun 8 2022 8:45AM,239068,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 8 2022 8:40AM,239066,19,"GCH Granules USA, Inc.",1.0,NaN,NaN
4,Jun 8 2022 8:28AM,239065,19,"GCH Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 8 2022 8:46AM,239069,10,"Amcyte Pharma, Inc.",2.0,0.0,0.0
1,Jun 8 2022 8:45AM,239067,10,ISDIN Corporation,26.0,0.0,0.0
2,Jun 8 2022 8:45AM,239068,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 8 2022 8:40AM,239066,19,"GCH Granules USA, Inc.",1.0,0.0,0.0
4,Jun 8 2022 8:28AM,239065,19,"GCH Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1912286,88.0,9.0,20.0
12,478048,1.0,2.0,0.0
16,239041,0.0,1.0,0.0
18,478026,2.0,0.0,0.0
19,1673270,47.0,1.0,0.0
20,1911881,83.0,75.0,3.0
21,1673244,7.0,0.0,0.0
22,956246,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1912286,88.0,9.0,20.0
1,12,478048,1.0,2.0,0.0
2,16,239041,0.0,1.0,0.0
3,18,478026,2.0,0.0,0.0
4,19,1673270,47.0,1.0,0.0
5,20,1911881,83.0,75.0,3.0
6,21,1673244,7.0,0.0,0.0
7,22,956246,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,88.0,9.0,20.0
1,12,1.0,2.0,0.0
2,16,0.0,1.0,0.0
3,18,2.0,0.0,0.0
4,19,47.0,1.0,0.0
5,20,83.0,75.0,3.0
6,21,7.0,0.0,0.0
7,22,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,88.0
1,12,Released,1.0
2,16,Released,0.0
3,18,Released,2.0
4,19,Released,47.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21,22
Status,,,,,,,,
Completed,20.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
Executing,9.0,2.0,1.0,0.0,1.0,75.0,0.0,1.0
Released,88.0,1.0,0.0,2.0,47.0,83.0,7.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21,22
0,Completed,20.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,9.0,2.0,1.0,0.0,1.0,75.0,0.0,1.0
2,Released,88.0,1.0,0.0,2.0,47.0,83.0,7.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21,22
0,Completed,20.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
1,Executing,9.0,2.0,1.0,0.0,1.0,75.0,0.0,1.0
2,Released,88.0,1.0,0.0,2.0,47.0,83.0,7.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()